# Level 2
# Client aggregates received 2 shares plus the one it kept and broadcasts solution

## Preparations

In [1]:
# modules
import socket  # creating server client connections for efficient communication
import pickle  # efficient file saving & loading
import time  # for stopping runtimes
import glob  # local file and directories listings 
import pandas as pd  # data mangling and transforming
import numpy as np  # matrix manipulations
from datetime import datetime  # for timestamps
from read_pickle import read_pickle  # binary file reading
from secret_sharing import decrypt  # import additive secret sharing decryption

In [ ]:
# start timing
start = time.time()

In [ ]:
# large prime number (key, hardcoded for now)
Q = 1006940912373947

In [ ]:
# hostnames & ips
file = open('data/hosts&ips.txt', 'r')
hoips = eval(file.read())
file.close()

### data share which was held

In [ ]:
# read local encrypted data
file = open('data/encryptedData.pickle', 'rb')
d = pickle.load(file)

In [ ]:
# only keep last column
share3 = d.C

### data shares which were received from other parties

In [ ]:
share1, share2 = glob.glob('data/[0-9]*-1.pickle')

In [ ]:
share1 = read_pickle(share1)
share2 = read_pickle(share2)

### Combine the 3

In [ ]:
shares = decrypt(*[share1,share2,share3], Q=Q)

In [ ]:
# save as pickle
with open('data/own_aggregated_share.pickle','wb') as f:
    pickle.dump(shares, f)

## Communication with other parties

### Establish connection to other parties (servers)

In [ ]:
# general options
s0 = socket.socket()  # Create socket object 0
s1 = socket.socket()  # Create socket object 1
client = socket.gethostname()  # define current machine as client
port = 60387  # Reserve a port for your service.

In [ ]:
# other parties as servers
servers = [key for key, value in hoips['hosts'].items() if value not in client]
server0 = hoips['hosts'][servers[0]]
server1 = hoips['hosts'][servers[1]]

In [ ]:
# double-check parties
print('Server0 ('+servers[0]+'): '+server0)
print('Server1 ('+servers[1]+'): '+server1)
print('Client: '+client)

In [ ]:
# connect
s0.connect((server0, port))
s1.connect((server1, port))

### Send data

In [ ]:
msg = pickle.dumps(shares)

In [ ]:
# send it
s0.send(msg)
s1.send(msg)

In [ ]:
# close sockets
s0.close()
s1.close()

In [ ]:
# timer
end = time.time()
df_timer = pd.DataFrame(data={'phase': '2: Aggregated Value Broadcasting',
                              'timestamp': str(datetime.utcnow()),
                              'n_secrets': len(d),
                              'seconds': [round(end-start,2)]})

with open('data/timer.csv', 'a') as f:
    df_timer.to_csv(f, header=False)